In [1]:
import spektral

C:\Users\Sidsavi\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Sidsavi\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Sidsavi\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
from spektral.datasets import citation

In [3]:
A = spektral.datasets.Citation('Cora')

C:\Users\Sidsavi\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:708: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self[i, j] = values


In [4]:
import networkx as nx

In [5]:
A[0]

Graph(n_nodes=2708, n_node_features=1433, n_edge_features=None, n_labels=7)

In [6]:
A[0].a

<2708x2708 sparse matrix of type '<class 'numpy.int32'>'
	with 10556 stored elements in Compressed Sparse Row format>

In [7]:
A[0].x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
A[0].e

In [9]:
from spektral.data import SingleLoader

In [10]:
Graph = SingleLoader(A)

In [11]:
Graph.load()

<RepeatDataset shapes: (((2708, 1433), (2708, 2708)), (2708, 7)), types: ((tf.float32, tf.int32), tf.int32)>

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from spektral.datasets.citation import Cora
from spektral.layers import GCNConv
from spektral.transforms import LayerPreprocess, AdjToSpTensor
from spektral.utils import tic, toc

tf.random.set_seed(seed=0)  # make weight initialization reproducible

# Load data
dataset = Cora(transforms=[LayerPreprocess(GCNConv), AdjToSpTensor()])
graph = dataset[0]
x, a, y = graph.x, graph.a, graph.y
mask_tr, mask_va, mask_te = dataset.mask_tr, dataset.mask_va, dataset.mask_te

# Define model
x_in = Input(shape=(dataset.n_node_features,))
a_in = Input((dataset.n_nodes,), sparse=True)
x_1 = Dropout(0.5)(x_in)
x_1 = GCNConv(16, 'relu', False, kernel_regularizer=l2(2.5e-4))([x_1, a_in])
x_2 = Dropout(0.5)(x_1)
x_2 = GCNConv(8, 'relu', False, kernel_regularizer=l2(2.5e-4))([x_2, a_in])
x_2 = Dropout(0.5)(x_2)
x_3 = GCNConv(y.shape[1], 'softmax', False)([x_2, a_in])

# Build model
model = Model(inputs=[x_in, a_in], outputs=x_3)
optimizer = Adam(lr=1e-2)
loss_fn = CategoricalCrossentropy()


# Training step
@tf.function
def train():
    with tf.GradientTape() as tape:
        predictions = model([x, a], training=True)
        loss = loss_fn(y[mask_tr], predictions[mask_tr])
        loss += sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


# Time the execution of 200 epochs of training
train()  # Warm up to ignore tracing times when timing
tic()
for epoch in range(1, 1001):
    loss = train()
toc('Spektral - GCN (1000 epochs)')
print(f"Final loss = {loss}")

C:\Users\Sidsavi\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:708: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self[i, j] = values


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the ver